In [113]:
import ctypes
import numpy as np
import copy
from scipy.signal import convolve2d

def sigmoid(x):
    return (1 / (1 + np.exp(-x)) )

def softmax(x):
    return np.exp(x) / sum(np.exp(x))

def getinputStackedColumns(inputMatrix, kernel_map_shape, stride):
    s0, s1, s2, s3 = inputMatrix.strides

    d1_input, d2_input, h_input, w_input = inputMatrix.shape
    h_kernel, w_kernel = kernel_map_shape

    out_shape = ( d1_input, d2_input, (h_input-h_kernel+1)//stride, (w_input-w_kernel+1)//stride, h_kernel, w_kernel)
    inputStackedColumns = np.lib.stride_tricks.as_strided(inputMatrix,
                                                          shape=out_shape,
                                                          strides=(s0, s1, stride*s2,stride*s3,s2, s3))
    return inputStackedColumns

def getPadding(h_kernel, w_kernel, h_input, w_input, stride):
    initial_h = h_input
    initial_w = w_input 
    
    while True:
        if (h_input-h_kernel+1)%stride!=0:
            h_input+=1
        else:
            break
            
    while True:
        w_input+=1
        if (w_input-w_kernel+1)%stride!=0:
            w_input+=1
        else:
            break

            
    return h_input-initial_h, w_input-initial_w

def Convolution_strided_img2col(inputMatrix, kernel_map, stride):
  
    d1_input, d2_input, h_input, w_input = inputMatrix.shape
    d_kernel, h_kernel, w_kernel = kernel_map.shape
    
    inputStackedColumns = getinputStackedColumns(inputMatrix, kernel_map.shape[1:], stride)
    
    out_shape = ( d1_input, d2_input, (h_input-h_kernel+1)//stride, (w_input-w_kernel+1)//stride, h_kernel, w_kernel)

    inputStackedColumns = inputStackedColumns.flatten()
    inputStackedColumns = np.reshape(inputStackedColumns, (d1_input, d2_input, (h_input-h_kernel+1)//stride * ((w_input-w_kernel+1)//stride), h_kernel*w_kernel ))
    kernel_map_edited = kernel_map.reshape(d_kernel, h_kernel*w_kernel).transpose()
    im2col_conv = np.einsum("ijkl,lm->jkm", inputStackedColumns, kernel_map_edited)
    im2col_conv = im2col_conv.swapaxes(0,2).swapaxes(1,2)
    im2col_conv = im2col_conv.reshape(im2col_conv.shape[0], im2col_conv.shape[1], out_shape[2], out_shape[3])
    return im2col_conv

def convolution(inputMatrix, kernel_map, stride):
    #d1_input, d2_input, h_input, w_input = inputMatrix.shape
    #d_kernel, h_kernel, w_kernel = kernel_map.shape

    #padding = getPadding(h_kernel, w_kernel, h_input, w_input, stride)
    #inputMatrix = np.pad(inputMatrix, ((0,0),(0,0),(0,padding[0]),(0,padding[1])))
    
    return Convolution_strided_img2col(inputMatrix, kernel_map, stride)


def Maxpooling(inputMatrix, kernel_shape, stride):

    d_kernel_map, d_input, h_input, w_input = inputMatrix.shape
    h_kernel, w_kernel = kernel_shape

    windows = getinputStackedColumns(inputMatrix, kernel_shape, stride)
    
    out_shape = ( d_kernel_map, d_input, (h_input-h_kernel+1)//stride, (w_input-w_kernel+1)//stride, h_kernel, w_kernel)

    maxs = np.max(windows, axis=(4,5))
    maxs = maxs.reshape(d_kernel_map, d_input, (h_input-h_kernel+1)//stride, (w_input-w_kernel+1)//stride)
    return maxs

def Averagepooling(inputMatrix, kernel_shape, stride):
        
    d_kernel_map, d_input, h_input, w_input = inputMatrix.shape
    h_kernel, w_kernel = kernel_shape
    
    windows = getinputStackedColumns(inputMatrix, kernel_shape, stride)
    
    out_shape = ( d_kernel_map, d_input, (h_input-h_kernel+1)//stride, (w_input-w_kernel+1)//stride, h_kernel, w_kernel)

    means = np.means(windows, axis=(4,5))
    means = means.reshape(d_kernel_map, d_input, (h_input-h_kernel+1)//stride, (w_input-w_kernel+1)//stride)
    return means

def pooling(inputMatrix, kernel_shape, stride, method):
    if method=="average":
        return Averagepooling(inputMatrix, kernel_shape, stride)
    
    elif method=="max":
        return Maxpooling(inputMatrix, kernel_shape, stride)

    else:
        return Maxpooling(inputMatrix, kernel_shape, stride)        

def RELU_Matrixoperation(inputMatrix):
    return np.maximum(inputMatrix, 0)

In [120]:
import numpy as np
import random as rd
from matplotlib import pyplot as plt
from matplotlib import image
import cv2
import csv
import os
import copy

class Model:
    
    imgShape = None
    convout1 = None 
    pooling1 = None
    relu1 = None
    convout2 = None 
    pooling2 = None
    relu2 = None
    x = None
    a2 = None
    a3 = None    
    z2 = None
    z3 = None
    
    w1 = None
    w2 = None 
    kernelmap1 = None
    kernelmap2 = None
    pooling1kernelshape = None
    pooling2kernelshape = None
    
    measurementData_Loss = []
    measurementData_Accuracy = []
    
    def __init__(self, inputArrayShape, n_a2=50, n_a3=2, d_kernel1=5, w_h_kernel1=5, d_kernel2=3, w_h_kernel2=3, poolingmethod="max", w_h_pooling1=2, w_h_pooling2=2, stride_conv1=1, stride_conv2=1, stride_pooling1=1, stride_pooling2=1, learningrate=0.1, batchsize=27):
        self.imgShape = copy.copy(inputArrayShape)
        self.poolingmethod = poolingmethod
        self.stride_conv1 = stride_conv1 
        self.stride_conv2 = stride_conv2 
        self.stride_pooling1 = stride_pooling1
        self.stride_pooling2 = stride_pooling2
        self.learningrate = learningrate
        self.batchsize = batchsize
        
        self.pooling1kernelshape = (w_h_pooling1,w_h_pooling1)
        self.pooling2kernelshape = (w_h_pooling2,w_h_pooling2)
        
        self.kernelmap1 = np.random.uniform(-1,1,(d_kernel1,w_h_kernel1,w_h_kernel1))
        self.kernelmap2 = np.random.uniform(-1,1,(d_kernel2,w_h_kernel2,w_h_kernel2))
        
        inputArrayShape.insert(0, 1)
        inputArrayShape.insert(0, 1) #input matrix: !one! example with depth !one!
        n_x = self.getNumberOutputNeuronsConvolutionalLayer(inputArrayShape)
        
        self.w1 = np.random.uniform(-1,1,(n_a2, n_x+1))
        self.w2 = np.random.uniform(-1,1,(n_a3, n_a2+1))
        
    def getNumberOutputNeuronsConvolutionalLayer(self, inputArrayShape):
        self.input = np.zeros((inputArrayShape[0], inputArrayShape[1], inputArrayShape[2], inputArrayShape[3]))
        self.__convolutional_layers(self.input)
        return (self.x.shape[0])
        
    def __convolutional_layers(self, input):
        self.convout1 = convolution(self.input, self.kernelmap1, self.stride_conv1) 
        self.reluout1 = RELU_Matrixoperation(self.convout1)
        self.poolingout1 = pooling(self.reluout1, self.pooling1kernelshape, self.stride_pooling1, self.poolingmethod)

        self.convout2 = convolution(self.poolingout1, self.kernelmap2, self.stride_conv2) 
        self.reluout2 = RELU_Matrixoperation(self.convout2)
        self.poolingout2 = pooling(self.reluout2, self.pooling2kernelshape, self.stride_pooling2, self.poolingmethod)
        
        self.x = self.poolingout2.reshape(self.poolingout2.shape[0], self.poolingout2.shape[1], self.poolingout2.shape[2]*self.poolingout2.shape[3])
        self.x = np.concatenate(self.x, axis=1).transpose()
        
    def propagateForward(self, input):
        self.input = input.reshape(input.shape[0], 1, input.shape[1], input.shape[2]).swapaxes(0,1)
        self.__convolutional_layers(self.input)
        self.x = np.pad(self.x, ((1,0),(0,0)), constant_values=1)
        
        self.z2 = np.dot(self.w1, self.x)
        self.a2 = sigmoid(self.z2)
        self.a2 = np.pad(self.a2, ((1,0),(0,0)), constant_values=1)
        
        self.z3 = np.dot(self.w2, self.a2)
        self.a3 = softmax(self.z3)
        
        return self.a3
        
        
    def propagateBackward(self, input, y_hat):
        self.propagateForward(input)
        pd_L_wrt_a3 = 1/input.shape[0] * (self.a3 - y_hat)
        pd_a3_wrt_z3 = softmax(self.z3)*(1 - softmax(self.z3))
        pd_z3_wrt_w2 = self.a2
        pd_L_wrt_w2 = np.dot(pd_L_wrt_a3 * pd_a3_wrt_z3, pd_z3_wrt_w2.T)
        
        pd_z3_wrt_a2 = self.w2
        pd_a2_wrt_z2 = sigmoid(self.z2)*(1 - sigmoid(self.z2))
        pd_z2_wrt_w1 = self.x
        pd_L_wrt_w1 = np.dot( np.dot((pd_L_wrt_a3 * pd_a3_wrt_z3).T, pd_z3_wrt_a2[:,1:]).T * pd_a2_wrt_z2, pd_z2_wrt_w1.T)

        self.w2 = self.w2 - self.learningrate * pd_L_wrt_w2
        self.w1 = self.w1 - self.learningrate * pd_L_wrt_w1
        
    def getDataBatch(self, path, filenames):
        uniform_size = self.imgShape[::-1]
        
        data_x = []
        data_y = []
        
        for i in filenames:
            
            img = image.imread(path + "//" + i)
            if len(img.shape)==3:
                img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
            resized_img = cv2.resize(img, dsize=uniform_size, interpolation=cv2.INTER_CUBIC)
            data_x.append(resized_img)


            y = None
            if "y" in i:
                y = np.array([1,0])
            else:
                y = np.array([0,1])
            data_y.append(y)
            
        data_x = np.array(data_x).astype(np.float)  
        data_y = np.array(data_y).astype(np.float).transpose()
        
        return(data_x, data_y)
        
    def train(self, epochs, trainpath, testpath):
        
        trainfilenames = os.listdir(trainpath)
        testfilenames = os.listdir(testpath)
        
        np.random.shuffle(trainfilenames)
        np.random.shuffle(testfilenames)
        
        batch_counter = 0
        for i in range(epochs):
            filenamesBatch = trainfilenames[batch_counter:batch_counter+self.batchsize]
            
            data = self.getDataBatch(trainpath, filenamesBatch)
            input = data[0]
            y_hat = data[1]
            self.propagateBackward(input, y_hat)
            loss = 1/input.shape[0] * ((self.a3 - y_hat)**2).sum()
            
            data_test = self.getDataBatch(testpath, testfilenames[:100])
            accuracy = self.test(data_test[0], data_test[1])
            print("Epoch " + str(i) + " done. Loss: " + str(loss) + " Accuracy: " + str(accuracy*100) + " %")
            self.measurementData_Accuracy.append(accuracy)
            self.measurementData_Loss.append(loss)
            
            batch_counter += self.batchsize
            if (batch_counter>=2700-self.batchsize):
                batch_counter = 0
            
    def test(self, input, y_hat):
        fx = self.propagateForward(input)
        number_correct = 0
        number_overall = fx[:,:100].shape[1]
        for i in range(number_overall):
            arrayfx = fx[:,i:i+1]
            arrayy = y_hat[:,i:i+1]
           
            indexmaxfx = np.argmax(arrayfx, axis=0)
            indexmaxy = np.argmax(arrayy, axis=0)
            if (indexmaxfx[0]==indexmaxy[0]):
                number_correct += 1
                
        return number_correct/number_overall
    
    def getMeasurementData(self):
        return (self.measurementData_Loss, self.measurementData_Accuracy)
        

In [ ]:
m = Model([175, 167], n_a2=1000, n_a3=2, d_kernel1=5, w_h_kernel1=5, d_kernel2=5, w_h_kernel2=5, poolingmethod="max", w_h_pooling1=3, w_h_pooling2=3, stride_conv1=2, stride_conv2=2, stride_pooling1=1, stride_pooling2=1, learningrate=0.5, batchsize=27)
m.train(500, "C://Users//Benni//Desktop//PK//Dateset//yes_and_no//train", "C://Users//Benni//Desktop//PK//Dateset//yes_and_no//test")

c:\users\benni\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:132: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
c:\users\benni\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:133: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
c:\users\benni\appdata\local\programs\python\python37\

Epoch 0 done. Loss: 0.9629629249187502 Accuracy: 45.0 %
Epoch 1 done. Loss: 0.9068388585705159 Accuracy: 45.0 %


In [ ]:
tracking_data_models = []
tracking_data_accuracy = []

while True:
    n_a2 = randint(1,50)*1000
    n_a3 = 2
    d_kernel1 = randint(1,10)*2
    w_h_kernel1 = randint(1,6)
    d_kernel1 = randint(1,10)*2
    w_h_kernel2 = randint(1,6)
    poolingmethod = randint(1,2) #1--> max, 2--> average
    w_h_pooling1 = randint(1,10)
    w_h_pooling2 = randint(1,10)
    stride_conv1 =
    stride_conv2 =
    stride_pooling1 =
    stride_pooling2 =
    learningrate = 0.5
    batchsize = 27
    
    m = Model([175, 167], n_a2, n_a3, d_kernel1, w_h_kernel1, d_kernel2, w_h_kernel2, poolingmethod=, w_h_pooling1, w_h_pooling2, stride_conv1, stride_conv2, stride_pooling1, stride_pooling2, learningrate, batchsize)
    m.train(500, "C://Users//Benni//Desktop//PK//Dateset//yes_and_no//train", "C://Users//Benni//Desktop//PK//Dateset//yes_and_no//test")